In [92]:
import pandas as pd

In [93]:
df = pd.read_csv("Filghts TEC_Valid.csv")

In [94]:
df['STD'] = pd.to_datetime(df['STD'])
df['STA'] = pd.to_datetime(df['STA'])

We remove the 2024 data.

In [95]:
df = df[df['STD'].dt.year < 2024]

We delete missing data.

In [113]:
df = df.dropna()

We remove outliers to keep only "real" flights.

In [97]:

Q1 = df["Passengers"].quantile(0.25)
Q3 = df["Passengers"].quantile(0.75)
IQR = Q3 - Q1

# Define the boundaries for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove rows where the column value is outside the boundaries
df_filtered1 = df[(df["Passengers"] >= lower_bound) & (df["Passengers"] <= upper_bound)]

We group by departure time and date, as well as by aircraft.

In [98]:

# Convert time columns to datetime
df_filtered1['STD'] = pd.to_datetime(df_filtered1['STD'])
df_filtered1['STA'] = pd.to_datetime(df_filtered1['STA'])

# Sort DataFrame by 'id' and 'departure_time'
df_filtered1 = df_filtered1.sort_values(by=['Aeronave', 'STD'])
df_filtered1.head(10)

/var/folders/1h/w3mjcszn60577hhjww35tglh0000gn/T/ipykernel_22512/1624798082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered1['STD'] = pd.to_datetime(df_filtered1['STD'])
/var/folders/1h/w3mjcszn60577hhjww35tglh0000gn/T/ipykernel_22512/1624798082.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered1['STA'] = pd.to_datetime(df_filtered1['STA'])


,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
221144,dd0a5be979063bbb5305bd3d50a3e785,9H-SWB,AW,BM,Ciudad Fronteriza,Ciudad Principal,2023-11-16 06:30:00,2023-11-16 07:35:00,180,192.0,120.0
160452,c2d8b0985485299dbbadc0dc23747f8b,9H-SWB,BM,AW,Ciudad Principal,Ciudad Fronteriza,2023-11-16 08:25:00,2023-11-16 13:10:00,180,168.0,95.0
189928,ba4bc41678ad4c0e392a027f2b843925,9H-SWB,AW,BD,Playa,Ciudad Principal,2023-11-16 20:20:00,2023-11-16 22:00:00,178,172.0,120.0
218790,788f7b866bcb230eb34f09d942afe73f,9H-SWB,AW,BM,Ciudad Fronteriza,Ciudad Principal,2023-11-17 06:30:00,2023-11-17 07:35:00,178,181.0,105.0
144055,83ac777580b704b361b4365c7a603dd5,9H-SWB,BM,AW,Ciudad Principal,Ciudad Fronteriza,2023-11-17 08:25:00,2023-11-17 13:10:00,178,179.0,127.0
226117,3024a466458bdae49a6e83f2a9f8eba3,9H-SWB,AW,BM,Ciudad Fronteriza,Ciudad Principal,2023-11-19 06:30:00,2023-11-19 07:35:00,178,179.0,105.0
149213,ac8b8bb7784534e2fe47f76a8de60e9c,9H-SWB,BM,AW,Ciudad Principal,Ciudad Fronteriza,2023-11-19 08:25:00,2023-11-19 13:10:00,178,176.0,96.0
189642,7de9829433de30fc20077515d50833b8,9H-SWB,AW,BD,Playa,Ciudad Principal,2023-11-19 20:20:00,2023-11-19 22:00:00,178,158.0,96.0
139955,8faac925e036a48e5809ddbd661cda02,9H-SWB,BD,AW,Ciudad Principal,Playa,2023-11-19 22:50:00,2023-11-20 00:25:00,178,188.0,119.0
229473,e1165c2cfce269543f8e1b6c285d2afd,9H-SWB,AW,BM,Ciudad Fronteriza,Ciudad Principal,2023-11-20 06:30:00,2023-11-20 07:35:00,178,180.0,101.0


In [99]:
# Shift the arrival_time backwards within each 'id' group
df_filtered1['nSTA'] = df_filtered1.groupby('Aeronave')['STA'].shift(1)

# Calculate duration in minutes between the current departure and the next arrival
df_filtered1['duration'] = -1*((df_filtered1['nSTA'] - df_filtered1['STD']).dt.total_seconds() / 60)

# Drop the last row of each group since it doesn't have a next arrival time
df_filtered1.dropna(subset=['duration'], inplace=True)

print(df_filtered1[['Aeronave', 'DepartureStation', 'ArrivalStation', 'STD', 'STA',"nSTA", 'duration']])

       Aeronave DepartureStation ArrivalStation                 STD  \
160452   9H-SWB               BM             AW 2023-11-16 08:25:00   
189928   9H-SWB               AW             BD 2023-11-16 20:20:00   
218790   9H-SWB               AW             BM 2023-11-17 06:30:00   
144055   9H-SWB               BM             AW 2023-11-17 08:25:00   
226117   9H-SWB               AW             BM 2023-11-19 06:30:00   
...         ...              ...            ...                 ...   
42723    XA-VYF               AT             AK 2023-12-31 06:15:00   
90203    XA-VYF               AK             AT 2023-12-31 10:25:00   
245128   XA-VYF               AT             BT 2023-12-31 12:50:00   
91086    XA-VYF               BT             AT 2023-12-31 16:55:00   
200462   XA-VYF               AT             BG 2023-12-31 19:15:00   

                       STA                nSTA  duration  
160452 2023-11-16 13:10:00 2023-11-16 07:35:00      50.0  
189928 2023-11-16 22:00:00 20

In [100]:
df_filtered1['flight'] = df['DepartureStation'] + '_' + df['ArrivalStation']

In [101]:
df_filtered1['nocargable'] = ~df_filtered1['DepartureStation'].isin(['AO', 'AU', 'AW', 'BA', 'BM', 'AT', 'AK', 'AD'])

In [102]:
df_filtered1.drop(columns=['DepartureStation', 'ArrivalStation', "Destination_Type", "Origin_Type","Capacity",	"Passengers",	"Bookings"], inplace=True)

In [103]:
df2 = pd.read_csv("dataset_combinado.csv")

In [104]:
combined_df = pd.merge(df_filtered1, df2, on='Flight_ID', how='inner')

We choose to work with Coca-Cola as an example, since sandwiches are rarely requested.

In [105]:
df = combined_df.loc[:, ["Aeronave",	"STD",	"STA",	"nSTA",	"duration", "flight","Coca Cola Regular", 'nocargable']]

We create a dictionary from the data to build a predictive model.

In [106]:
df1 = df[df["Aeronave"]=="XA-VYF"]

In [107]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 473 entries, 108174 to 108646
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Aeronave           473 non-null    object        
 1   STD                473 non-null    datetime64[ns]
 2   STA                473 non-null    datetime64[ns]
 3   nSTA               473 non-null    datetime64[ns]
 4   duration           473 non-null    float64       
 5   flight             473 non-null    object        
 6   Coca Cola Regular  473 non-null    float64       
 7   nocargable         473 non-null    bool          
dtypes: bool(1), datetime64[ns](3), float64(2), object(2)
memory usage: 30.0+ KB


In [115]:
df1['day'] = df1['STD'].dt.date
df1['flight_number'] = df1.groupby('day').cumcount() + 1

In [121]:
df1.drop(columns=["STD","STA","nSTA"] , inplace=True)

# Final Model

This is a sample model for one aircraft type, one item type, the load capacity per trip, and the airports where loading is allowed. The model is flexible and can be adapted for other parameters.

The model groups by day and flight number. First, it checks whether loading is allowed at that location. If so, it verifies demand and the loading plan. If restocking is needed, inventory is updated. To determine if restocking is required that day, it checks the current inventory and the product’s expiration. If the expiration has passed, restocking is triggered; otherwise, the inventory is just updated.

In [125]:
def optimize_coca_cola_supply(data):
    #Se agrupan por dia y numero de vuelo
    data = data.sort_values(by=['day', 'flight_number'])

    supply_dates = {}
    current_stock = 0
    last_supply_day = None

    for index, row in data.iterrows():
        if row['nocargable']:
            continue  # No se revisa si se debe cargar en los stops que no se peudan cargar

        # Se debe determinar la demanda y el plan de carga
        if current_stock < row['Coca Cola Regular']:
            # Se debe checar si se debe cargar
            if last_supply_day is None or (pd.to_datetime(row['day']) - last_supply_day).days >= 2:
                # Se hace la carga y se refreseha el inventario
                supply_dates[(row['day'], row['flight_number'])] = 'Supply'
                current_stock = 24 - row['Coca Cola Regular']  # Se calcula el inventario
                last_supply_day = pd.to_datetime(row['day'])
            else:
                # Consider the remaining lifespan of the stock
                days_since_supply = (pd.to_datetime(row['day']) - last_supply_day).days
                if days_since_supply < 2:
                    # Use existing stock
                    current_stock -= row['Coca Cola Regular']
                else:
                    # Stock expired, need new supply
                    supply_dates[(row['day'], row['flight_number'])] = 'Supply'
                    current_stock = 24 - row['Coca Cola Regular']  # Supply new and consume some
                    last_supply_day = pd.to_datetime(row['day'])
        else:
            # Sufficient stock to meet demand
            current_stock -= row['Coca Cola Regular']

    return supply_dates

# Apply the optimization function
supply_decisions = optimize_coca_cola_supply(df1)
supply_decisions


{(datetime.date(2023, 9, 7), 1): 'Supply',
 (datetime.date(2023, 9, 9), 1): 'Supply',
 (datetime.date(2023, 9, 11), 1): 'Supply',
 (datetime.date(2023, 9, 13), 1): 'Supply',
 (datetime.date(2023, 9, 15), 1): 'Supply',
 (datetime.date(2023, 9, 17), 1): 'Supply',
 (datetime.date(2023, 9, 19), 1): 'Supply',
 (datetime.date(2023, 9, 21), 1): 'Supply',
 (datetime.date(2023, 9, 23), 1): 'Supply',
 (datetime.date(2023, 9, 25), 1): 'Supply',
 (datetime.date(2023, 9, 27), 1): 'Supply',
 (datetime.date(2023, 9, 29), 1): 'Supply',
 (datetime.date(2023, 10, 1), 2): 'Supply',
 (datetime.date(2023, 10, 3), 1): 'Supply',
 (datetime.date(2023, 10, 5), 1): 'Supply',
 (datetime.date(2023, 10, 7), 1): 'Supply',
 (datetime.date(2023, 10, 9), 2): 'Supply',
 (datetime.date(2023, 10, 11), 1): 'Supply',
 (datetime.date(2023, 10, 13), 1): 'Supply',
 (datetime.date(2023, 10, 15), 1): 'Supply',
 (datetime.date(2023, 10, 18), 3): 'Supply',
 (datetime.date(2023, 10, 20), 1): 'Supply',
 (datetime.date(2023, 10, 22)

In [135]:
formatted_decisions = [f"Día {day}: En el vuelo {flight} se tiene que cargar artículo." for (day, flight) in supply_decisions.keys()]

# Joining all formatted strings for easy viewing
formatted_output = "\n".join(formatted_decisions)
print(formatted_output)

Día 2023-09-07: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-09: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-11: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-13: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-15: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-17: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-19: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-21: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-23: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-25: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-27: En el vuelo 1 se tiene que cargar artículo.
Día 2023-09-29: En el vuelo 1 se tiene que cargar artículo.
Día 2023-10-01: En el vuelo 2 se tiene que cargar artículo.
Día 2023-10-03: En el vuelo 1 se tiene que cargar artículo.
Día 2023-10-05: En el vuelo 1 se tiene que cargar artículo.
Día 2023-10-07: En el vuelo 1 se tiene que cargar artículo.
Día 2023-10-09: En el vuelo 2 se tiene q